# Kemal Demirel
# 191104091

In [57]:
import cv2
from PIL import Image
import os
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine


In [35]:
def preprocess_image(image):
    image = cv2.resize(image, (300,300) )
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # For RGB
    return image

In [37]:
def traverse_folder_to_preprocess(path):
    images = []
    for file in os.listdir(path):
        image = cv2.imread(os.path.join(path, file))
        image = preprocess_image(image)
        images.append(image)
    return images

In [38]:
# Pokemon resimlerinin bulunduğu path
pokemon_path = os.getcwd()+"/pokemon/"
pokemon_preprocessed_images = traverse_folder_to_preprocess(pokemon_path)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

In [27]:
# QI resimlerinin bulunduğu path
qi_path = os.getcwd()+"/QI/"
qi_preprocessed_images = traverse_folder_to_preprocess(qi_path)

121.png


error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [50]:
sift = cv2.xfeatures2d.SIFT_create()

[ WARN:0@6541.628] global shadow_sift.hpp:15 SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


In [51]:
dataset_features = []

for image in pokemon_preprocessed_images:
    kp, desc = sift.detectAndCompute(image, None)
    dataset_features.append(desc)
    
dataset_features = np.vstack(dataset_features)

In [54]:
num_clusters = 100
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(dataset_features) # random state silmeyi dene olmazsa

In [55]:
dataset_histograms = []
for image in pokemon_preprocessed_images:
    kp, desc = sift.detectAndCompute(image, None)
    labels = kmeans.predict(desc)
    hist, _ = np.histogram(labels, bins=num_clusters, range=(0, num_clusters-1))
    hist = hist.astype(float) / np.sum(hist)
    dataset_histograms.append(hist)
dataset_histograms = np.vstack(dataset_histograms)

In [ ]:
query_histograms = []
for image in qi_preprocessed_images:
    kp, desc = sift.detectAndCompute(image, None)
    labels = kmeans.predict(desc)
    hist, _ = np.histogram(labels, bins=num_clusters, range=(0, num_clusters-1))
    hist = hist.astype(float) / np.sum(hist)
    query_histograms.append(hist)
query_histograms = np.vstack(query_histograms)

In [ ]:
def find_similar_image(input_image):
    input_image = cv2.resize(input_image, (300,300))
    gray_input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    kp, desc = sift.detectAndCompute(gray_input_image, None)
    labels = kmeans.predict(desc)
    query_hist, _ = np.histogram(labels, bins=k, range=(0, k-1))
    query_hist = query_hist.astype(float) / np.sum(query_hist)
    similarities = []
    for hist in histograms:
        similarity = 1 - cosine(query_hist, hist)
        similarities.append(similarity)
    return similarities


In [ ]:
vocabulary = kmeans.cluster_centers_

In [ ]:
similarities = []
for query_hist in query_histograms:
    similarity = cosine_similarity(query_hist.reshape(1, -1), dataset_histograms)
    similarities.append(similarity[0])

# Rank the dataset images by their similarity to each query image and display the top 3
for i in range(len(query_images)):
    query_img = query_images[i]
    similarity_scores = similarities[i]
    ranked_indices = np.argsort(similarity_scores)[::-1][:3]
    ranked_images = [dataset_images[idx] for idx in ranked_indices]

    # Display the query image and the 3 most similar images
    cv2.imshow("Query Image", query_img)
    for j in range(len(ranked_images)):
        ranked_img = ranked_images[j]
        cv2.imshow("Similar Image {}".format(j+1), ranked_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()